In [2]:
# Initialize PaddleOCR instance
from sklearn.cluster import KMeans
import json
import numpy as np
from scipy.spatial import distance_matrix
data = json.load(open('output/img_res.json', 'r', encoding='utf-8'))


    


In [78]:
km = KMeans(n_clusters=4, random_state=0)
x_left = [poly[0][0] for poly in data['rec_polys']]
y_left = [poly[0][1] for poly in data['rec_polys']]
x_left = np.array(x_left).reshape(-1, 1)
km.fit_predict(x_left)



array([0, 2, 3, 1, 0, 0, 3, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 2, 0, 1, 2, 3,
       0, 2, 3, 1, 0, 1, 3, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1,
       2, 3, 0, 1, 2, 3, 0, 2, 3, 1, 0, 1, 2, 3, 3, 0, 1, 2, 3, 0, 1, 2,
       3, 0, 1, 2, 3, 0, 1, 2, 3, 3, 0, 1, 2, 2, 3, 1, 3, 0, 1, 2, 3, 0,
       1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 2, 3, 0, 1, 3, 0, 1, 2, 0, 1, 2, 3,
       3, 0, 1, 2, 2, 1, 0, 2, 3, 2, 3, 0, 1, 1, 2, 3, 0, 1, 2, 3, 0, 3,
       0, 1, 2, 3, 0, 1, 1, 2, 3, 0, 1, 3, 0, 1, 2, 3, 0, 2, 3, 1, 0, 1,
       2, 3, 0, 1, 2, 3, 1, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2,
       0, 1, 1, 2, 3, 0, 1, 1, 2, 3, 0, 1, 1, 1, 2, 0, 1, 3, 1, 2, 0, 1,
       2, 0, 1, 1, 2, 3, 0, 1, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3,
       0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0],
      dtype=int32)

In [89]:
text = [[] for _ in range(4)]
boxes = [[] for _ in range(4)]
for idx, j in enumerate(km.labels_):
        text[j].append(data['rec_texts'][idx])
        boxes[j].append(data['rec_polys'][idx])
# order clusters left to right
cluster_centers = km.cluster_centers_.flatten() 
sorted_indices = np.argsort(cluster_centers)
text = [text[i] for i in sorted_indices]
boxes = [boxes[i] for i in sorted_indices]

#sort text and boxed top to bottom within each cluster 
for i in range(4):
    y_coords = [box[0][1] for box in boxes[i]]
    sorted_indices = np.argsort(y_coords)
    text[i] = [text[i][j] for j in sorted_indices]
    boxes[i] = [boxes[i][j] for j in sorted_indices]



            


In [90]:
import re
lection_number_pattern = re.compile(r'\d.')
for i in range(4):
    for j in range(len(text[i])):
        text[i][j] = re.sub(lection_number_pattern, '', text[i][j]).strip() 



In [91]:

# match text by min vertical distance between their boxes
box_0_centers = [b[0][1] + b[3][1] / 2 for b in boxes[0]]
box_1_centers = [b[0][1] + b[3][1] / 2 for b in boxes[1]]
box_2_centers = [b[0][1] + b[3][1] / 2 for b in boxes[2]]
box_3_centers = [b[0][1] + b[3][1] / 2 for b in boxes[3]]

In [ ]:
# merge same line elements
def merge(box, text):
    merged_box = []
    merged_text = []
    avg_dist = np.median((np.asarray(box)-np.roll(box, 1))[1:])
    while len(box) > 0:
        b0, t0 = box.pop(), text.pop()
        if len(box) > 0:
            b1, t1 = box.pop(), text.pop()
        else:
            merged_box.insert(0,b0)
            merged_box.insertd(0,b1)
            return merged_box, merged_text
        
        print(np.abs(b0-b1))
        if np.abs(b0-b1) < avg_dist//4:
            merged_box.insert(0,np.mean([b0, b1]))
            merged_text.insert(0," ".join([t0,t1]))
        else:
            merged_box.insert(0,b0)
            merged_box.insert(0,b1)
            merged_text.insert(0,t0)
            merged_text.insert(0,t1)
    
    return merged_box, merged_text

box_0, text0 = merge(box_0_centers.copy(), text[0].copy())


78.5


TypeError: 'float' object cannot be interpreted as an integer

In [99]:
text0

['alla stracciatella',
 'Cin cin!',
 'per',
 'il gin',
 'Genova',
 'lo scioglilingua',
 'la pausa',
 'la pausa caffe',
 'lag gondola',
 'cappuccino il',
 'il latte macchiato',
 'il',
 'lago di Garda',
 '',
 'il cioccolato',
 'ripetete inf ripetere ',
 'ilt fungo pl funghi',
 'gli spaghetti pl',
 'la laguna',
 "il giro d'Italia",
 'il vulcano',
 ' la musica',
 'Via!',
 'Pronti?',
 'la risposta pl risposte',
 'la domanda pl domande Frage',
 'Firenze',
 'il/la professore/-essa',
 'studenti/studentesse',
 'lo/la studente/-essa pl',
 'famoso/a pl-i-e',
 'Roma',
 'di',
 'sono inf essere',
 'loro',
 'noi',
 'lei',
 'essere',
 'tocca a noi',
 'insomma',
 'abbastanza',
 'non',
 "non c'e male",
 'no?',
 'Vero?',
 'il dialogo pl-ghi',
 'leggete inf leggere*',
 'Milano',
 'a',
 'sto bene',
 'piacere',
 'sì',
 'il signore',
 'il signor...',
 'lui',
 'no',
 'sei inf essere*',
 'siamo infessere',
 'siete inf essere*',
 'chi?']

In [69]:
def fifo_match(box0, text0, box1, text1):
    avg_dist = np.median((np.asarray(box0)-np.roll(box0, 1))[1:])
    results = []
    b0, it = box0.pop(0), text0.pop(0)
    b1, de = box1.pop(0), text1.pop(0)
    while True:
        if np.abs(b0 - b1) < avg_dist//2:
            results.append([[it],[de]])
            b0, it = box0.pop(0), text0.pop(0)
            b1, de = box1.pop(0), text1.pop(0)

        elif b0 < b1:
            results[-1][0].append(it)
            if len(box0) > 0:
                b0, it = box0.pop(0), text0.pop(0)
            else:
                b1, de = box1.pop(0), text1.pop(0)
 
        elif b0 > b1:
            results[-1][1].append(de)
            if len(box1) > 0:
                b1, de = box1.pop(0), text1.pop(0)
            else:
                b0, it = box0.pop(0), text0.pop(0)

        if len(box0) == 0 and len(box1) == 0:
            for idx, (it, de) in enumerate(results):
                if len(it) > 1:
                    results[idx][0] = "<br>".join(it)
                if len(de) > 1:
                    results[idx][1] = "<br>".join(de)
            return results
results = fifo_match(box_0_centers.copy(), text[0].copy(), box_1_centers.copy(), text[1].copy()) 
 
            

In [70]:
[x for x in results]


[[['voi'], ['ihr; Sie']],
 [['chi?'], ['wer?']],
 [['siete inf essere*'], ['ihr seid; Sie sind']],
 [['siamo infessere'], 'wir sind<br>。'],
 [['sei inf essere*'], ['du bist']],
 [['no'], ['nein']],
 [['lui'], ['er']],
 [['il signor...'], ['Herr ...']],
 [['il signore'], ['Herr']],
 [['sì'], ['ja']],
 [['piacere'], ['freut mich']],
 [['sto bene'], ['es geht mir gut']],
 [['a'], ['in']],
 [['Milano'], ['Mailand']],
 [['leggete inf leggere*'], ['lesen Sie']],
 [['il dialogo pl-ghi'], ['Dialog']],
 [['Vero?'], ['nicht wahr?']],
 [['no?'], ['oder?']],
 [["non c'e male"], ['nicht schlecht']],
 [['non'], ['nicht']],
 [['abbastanza'], ['ganz, ziemlich']],
 [['insomma'], ['na ja']],
 [['tocca a noi'], ['wir sind dran']],
 [['essere'], ['sein']],
 [['lei'], ['sie ( Person Singular)']],
 [['noi'], ['wir']],
 [['loro'], ['sie ( Person Plural)']],
 [['sono inf essere'], 'sie sind<br>*'],
 [['di'], ['aus']],
 [['Roma'], ['Rom']],
 [['famoso/a pl-i-e'], ['berühmt']],
 ['lo/la studente/-essa pl<br>stu

In [38]:
[b for b in text[0]]

['voi',
 'chi?',
 'siete inf essere*',
 'siamo infessere',
 'sei inf essere*',
 'no',
 'lui',
 'il signor...',
 'il signore',
 'sì',
 'piacere',
 'sto bene',
 'a',
 'Milano',
 'leggete inf leggere*',
 'il dialogo pl-ghi',
 'Vero?',
 'no?',
 "non c'e male",
 'non',
 'abbastanza',
 'insomma',
 'tocca a noi',
 'essere',
 'lei',
 'noi',
 'loro',
 'sono inf essere',
 'di',
 'Roma',
 'famoso/a pl-i-e',
 'lo/la studente/-essa pl',
 'studenti/studentesse',
 'il/la professore/-essa',
 'Firenze',
 'la domanda pl domande Frage',
 'la risposta pl risposte',
 'Pronti?',
 'Via!',
 'la musica',
 '',
 'il vulcano',
 "il giro d'Italia",
 'la laguna',
 'gli spaghetti pl',
 'fungo pl funghi',
 'ilt',
 '',
 'ripetete inf ripetere',
 'cioccolato',
 '',
 'il',
 'lago di Garda',
 'il',
 'il latte macchiato',
 'il',
 'cappuccino',
 'gondola',
 'lag',
 'la pausa caffe',
 'la pausa',
 'lo scioglilingua',
 'Genova',
 'il gin',
 'per',
 'Cin cin!',
 'alla stracciatella']

In [39]:

[b for b in text[1]]

['ihr; Sie',
 'wer?',
 'ihr seid; Sie sind',
 'wir sind',
 '。',
 'du bist',
 'nein',
 'er',
 'Herr ...',
 'Herr',
 'ja',
 'freut mich',
 'es geht mir gut',
 'in',
 'Mailand',
 'lesen Sie',
 'Dialog',
 'nicht wahr?',
 'oder?',
 'nicht schlecht',
 'nicht',
 'ganz, ziemlich',
 'na ja',
 'wir sind dran',
 'sein',
 'sie ( Person Singular)',
 'wir',
 'sie ( Person Plural)',
 'sie sind',
 '*',
 'aus',
 'Rom',
 'berühmt',
 'Student/in',
 'Professor/in',
 'Florenz',
 'Antwort',
 'Fertig?',
 'Los!',
 'Musik',
 'Vulkan',
 "Giro d'Italia",
 'Lagune',
 'Spaghetti',
 'Pilz',
 'wiederholen Sie',
 'Schokolade',
 'Gardasee',
 'Latte macchiato',
 'Cappuccino',
 'Gondel',
 'Kaffeepause',
 'Pause',
 'Zungenbrecher',
 'Genua',
 'Gin',
 'für',
 'Prost!',
 'Stracciatella-']